# Mistral Guanoca LoRA PEFT example

Source Article: https://www.datacamp.com/tutorial/mistral-7b-tutorial

In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 10.0 MB/s eta 0:00:0000:010:01
  Using cached peft-0.9.0-py3-none-any.whl.metadata (13 kB)
Using cached peft-0.9.0-py3-none-any.whl (190 kB)
  Using cached tyro-0.7.3-py3-none-any.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 7.4 MB/s eta 0:00:00
Using cached tyro-0.7.3-py3-none-any.whl (79 kB)


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name="mistralai/Mistral-7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                          device_map="auto"
                                         )
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
print(tokenizer.add_bos_token, tokenizer.add_eos_token)

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]


model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

def chat(messages):
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    
    model_inputs = encodeds.to(torch.cuda.current_device())
    
    generated_ids = model.generate(model_inputs, max_new_tokens=2000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(encodeds[0])
    print(decoded[0])
    return decoded[0]

chat(messages)

/home/user/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.41s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


True True


2024-03-20 21:02:55.278734: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 21:02:55.278768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 21:02:55.279807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 21:02:55.284765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 21:02:55.811000: W tensorflow/compiler/tf2

tensor([    1,   733, 16289, 28793,  1824,   349,   574, 16020,  2076,  2487,
        28804,   733, 28748, 16289, 28793,  6824, 28725,   315, 28742, 28719,
         3448, 10473,   298,   264,  1179, 11322, 19961,   302,  6138, 23598,
        18342, 28723,   661, 13633,   776,   272,  1103,  3558,   302,   686,
        16944, 15637,   423,   298,  5681,   315, 28742, 28719, 13198,   582,
          297,   272,  6132, 28808,     2, 28705,   733, 16289, 28793,  2378,
          368,   506,   993,  7136,   864, 21116, 28804,   733, 28748, 16289,
        28793])
<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a simple recipe for a classic mayonnaise that you can whip up at home:

Ingredients:

* 2 large egg yolks
* 1 tablespoon Dijon mustard
* 2 tablespoons white wine 

"<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s>  [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a simple recipe for a classic mayonnaise that you can whip up at home:\n\nIngredients:\n\n* 2 large egg yolks\n* 1 tablespoon Dijon mustard\n* 2 tablespoons white wine vinegar\n* 1/2 cup vegetable oil\n* Salt and pepper to taste\n\nInstructions:\n\n1. In a small bowl, whisk together the egg yolks, Dijon mustard, and white wine vinegar until well combined.\n2. Slowly drizzle in the vegetable oil while whisking continuously.\n3. Continue whisking until the mixture thickens and emulsifies.\n4. Season to taste with salt and pepper.\n\nThat's it! Once you've mastered the basic mayonnaise recipe, you can experiment with different variations, like adding herbs or spices for a unique twist. And if you want to make an aiol

In [2]:
from datasets import load_dataset
import pandas as pd 

# df=pd.read_csv('gemma1000_w7b.csv')

# df.columns
ds = load_dataset("csv",data_files={"train":'gemma1000_w7b.csv'})

In [3]:
# df=df[['original_text','gemma_7b_rewritten_text_temp0','rewrite_prompt']]
# df['diff_prompt']='Tell me what prompt converted the following text: \n"""' + df['original_text'] + '"""\n to this text \n"""' + df['gemma_7b_rewritten_text_temp0'] + '"""\n'
#original text prefix
import json

orig_prefix = "Original Text: "

#mistral "response"
llm_response_for_rewrite = "Provide the modified text and I'll tell you a something general that changed about it.  I'll avoid any specifics though.  My tone will be neutral."

#modified text prefix
rewrite_prefix = "Rewritten Text: "

#provided as start of Mistral response (anything after this is used as the prompt)
#providing this as the start of the response helps keep things relevant
response_start = "The request was: "

#We insert our detected prompt into this well-scoring baseline text
#thanks to: https://www.kaggle.com/code/rdxsun/lb-0-61
base_line = 'Refine the following passage by emulating the writing style of [insert desired style here], with a focus on enhancing its clarity, elegance, and overall impact. Preserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style.Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.' 
base_line_swap_text = "[insert desired style here]"

# this function is used to output the right formate for each row in the dataset
def create_text_row(instruction, output) -> str:
    if output is not None:
        text_row = f"""<s>[INST] {instruction} [/INST] \\n {response_start} {output} </s>"""
    else:
        text_row = f"""<s>[INST] {instruction} [/INST] \\n {response_start}"""
    return text_row


def get_prompt(orig_text, transformed_text, rewritten_text = None):
    if rewritten_text is None:
        message = create_text_row(f"{orig_prefix} {orig_text}",llm_response_for_rewrite) + create_text_row(f" {rewrite_prefix} {transformed_text}", None)
    else:
        message = create_text_row(f"{orig_prefix} {orig_text}",llm_response_for_rewrite) + create_text_row(f" {rewrite_prefix} {transformed_text}",{rewritten_text})
    return message


# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for item in ds["train"]:
            json_object = {
                "text": get_prompt(item["original_text"],item['gemma_7b_rewritten_text_temp0'],item['prompt'])
            }
            output_jsonl_file.write(json.dumps(json_object) + "\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset.jsonl")


In [4]:
train_dataset = load_dataset('json', data_files='./training_dataset.jsonl' , split='train')

Generating train split: 1000 examples [00:00, 97535.15 examples/s]


In [5]:
# def chat(x):
#     model_inputs = tokenizer.apply_chat_template(x, return_tensors="pt")    
#     generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
#     decoded = tokenizer.batch_decode(generated_ids)
#     return decoded

# for i,r in df.iterrows():
#     message=[
#     {"role": "user", "content": r['original_text']},
#     {"role": "assistant", "content": r[
#     ]
#     df.at[i, 'predicted_prompt']=chat(message)
#     print(df.at[i, 'predicted_prompt'])
#     if (i>5):
#         break

# df.iloc[0:6,]

In [6]:
from transformers import HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os
from datasets import load_dataset
from trl import SFTTrainer

In [7]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head",]
)
model = get_peft_model(model, peft_config)

In [8]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_total_limit = 2, 
    save_strategy = "no", 
    load_best_model_at_end=False,
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant"
)

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/home/user/envs/test_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5462.92 examples/s]
/home/user/envs/test_env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
new_model='Mistral-7b-instruct-v0.1-qlora'
#new_model='Mistral-7b-v0.1-qlora'
trainer.train()
trainer.model.save_pretrained(new_model)


/home/user/envs/test_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.086300
50,0.952100
75,0.941300
100,0.884100
125,0.908300
150,0.882400
175,0.889700
200,0.886100
225,0.914900
250,0.884200


/home/user/envs/test_env/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [11]:
# Merge the model with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16, #bfloat and float?
    device_map="auto",
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.47s/it]
/home/user/envs/test_env/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [12]:
logging.set_verbosity(logging.CRITICAL)

prompt = "How do I find true love?"
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/user/envs/test_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/user/envs/test_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] How do I find true love? [/INST] Finding true love is a journey that requires time, effort, and self-discovery. Here are some steps to help you on your journey:

1. Focus on yourself: Before you can find true love, you need to focus on yourself and your own happiness. This means taking care of your physical, emotional, and mental well-being.
2. Set boundaries: It's important to set boundaries for yourself and communicate them clearly to potential partners. This includes setting boundaries for your time, your space, and your expectations.
3. Be open to new experiences: Don't be afraid to try new things and meet new people. This can help you expand your horizons and increase your chances of finding true love.
4. Be patient: Finding true love takes time, so be patient and don't rush the process.
5. Be honest and authentic: Be yourself


In [13]:
prompt = "What is Datacamp Career track?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is Datacamp Career track? [/INST] The Datacamp Career Track is a program that provides a comprehensive learning path for individuals who want to become data scientists. It includes a series of courses, projects, and assessments that cover a wide range of topics in data science, including statistics, programming, machine learning, and data visualization.

The program is designed to be flexible and self-paced, allowing learners to progress at their own speed. It also includes a variety of interactive elements, such as quizzes, coding challenges, and projects, to help learners apply their knowledge and skills in practical ways.

Upon completion of the program, learners will receive a certificate from Datacamp, which can be used to demonstrate their proficiency in data science to potential employers. The program is also designed to prepare learners for the Data Science Certification exam offered by the International Institute of Analytics (IIA).



In [14]:
prompt = "What is stochastic gradient descent?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is stochastic gradient descent? [/INST] Stochastic gradient descent (SGD) is an optimization algorithm commonly used to train machine learning models. It is a variation of the standard gradient descent algorithm, which is used to minimize the cost function of a model by iteratively adjusting its parameters.

The key difference between standard gradient descent and stochastic gradient descent is the way the gradient is calculated. In standard gradient descent, the gradient is calculated using the entire training set, which can be computationally expensive, especially for large datasets. In contrast, stochastic gradient descent calculates the gradient using a random subset of the training data, which is typically much smaller and faster to compute.

This allows SGD to converge more quickly and efficiently than standard gradient descent, especially for large datasets. Additionally, SGD is less sensitive to the presence of outliers in the data, which can help improve the sta

In [15]:
from random import randrange
sample = ds["train"][randrange(len(train_dataset ))]

prompt = get_prompt(sample["original_text"],sample['gemma_7b_rewritten_text_temp0'])

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.5)

print(f"Prompt:\n{prompt}\n")
print(f"\nGenerated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"\nGround truth:\n{sample['rewrite_prompt']}")

Prompt:
<s>[INST] Original Text:  Panhandler Lawsuit Kim Pindak stands outside Chicago's federal courthouse after the first day of testimony in a civil lawsuit involving him and and other panhandlers Monday, Jan. 25, 2016, in Chicago. He and another panhandler brought a suit that alleges they lost up to $10 a day because authorities bar them from begging in Daley Plaza, a popular public square in the city. The trial that started today is a rare instance of jurors being asked to decide the issue of panhandlers' rights. (AP Photo/Michael Tarm) CHICAGO -- A federal civil trial pitting two panhandlers against one of the nation's [/INST] \n The request was:  Provide the modified text and I'll tell you a something general that changed about it.  I'll avoid any specifics though.  My tone will be neutral. </s><s>[INST]  Rewritten Text:  ## Public Service Announcement

**Attention, Chicago residents:**

This public service announcement brings awareness to a ongoing case involving panhandlers an